In [ ]:
# Evaluating Results for HomeLocation prediction GeoLocation Models
# geoLocText ,  geoLocHybrid , geoLocLP( results has brackets)

import csv
from geopy.distance import distance

model="geoLocMOD"


# Read all labels ( training + test) and save to dictionary dLabels
# Read results and save to dictionary dResults

# Path ground truth data
pathLabel= "C:/Users/dennis/Desktop/dataTestTrain/userLabel.csv"

# Path predictions
pathResults="C:/Users/dennis/Desktop/geoLocLP.csv"


dLabels={}
with open(pathLabel,encoding="utf8") as labels:
    for line in labels:
        data = line.split(";")
        user=data[0]
        label=str(data[1])
        dLabels[user]=label
        
dResults={}
with open (pathResults, encoding="utf8") as results:
    for line in results:
        data = line.split(";")
        user=data[0]
        predict=str(data[1])

        if model=="geoLocLP" or "geoLocHybrid":
            predict= predict.replace("[","")
            predict= predict.replace("]","")

        dResults[user]=predict  
    
# Loop for dResults
#search key in dLabels key, and save dLabel[key]
#calculate with method distance between dResults[key] and dLabel[key]
#save calculated distance in list 

distances=[]

for user,predict in dResults.items():
    if user in dLabels.keys():
        label=dLabels[user]
        distances.append(int(round(distance(label,predict).km)))
          

lDistances = sorted(distances)
length=len(distances)
index=int(length/2)


print(model+" results:")
avg  = int ( sum(distances)/len(distances))
print("Mean is "+str(avg)+"km")
median = lDistances[index]
print("Median is "+str(median)+"km")
acc_at_161 = 100 * len([d for d in lDistances if d < 161]) / float(len(lDistances))
print("acc@161 is "+str(acc_at_161))

In [ ]:
# Calculate accuracy for DeepGeo models
# Exctract geo-tagged Tweets for ground truth data
import json
import csv
import reverse_geocoder as rg

hydrated = "C:/Users/dennis/Desktop/hydrated.jsonl"

data ={}
dUserCoordinates ={}

labels={}
with open(hydrated,encoding="utf8",errors='ignore') as hydratedJSONL:

    for raw in hydratedJSONL:      
        tweet = json.loads(raw)
        tweetID = tweet["id_str"] 

        
        geo = tweet["geo"]
        if geo != None:
            coordinates = tweet["geo"]["coordinates"]
            labels[tweetID]=coordinates
            continue
               
        if (tweet["place"]!=None) and geo == None  :
            if tweet["place"]["bounding_box"] != None:               
                #Coordinates have to be swapped from long,lat -> lat,long
                coordinates = list(tweet["place"]["bounding_box"]["coordinates"][0][0])
                coordinates = (coordinates[1],coordinates[0])
                labels[tweetID]=coordinates
                
print("Done with extracting Geo-Tagged Tweets")


import csv
from geopy.distance import distance

# Calculate accuracy of DeepGeo predictions

# Path predictions of DeepGeo models
pathResults="C:/Users/dennis/Desktop/dataTestTrain/results/deepGeoUserHL.csv"


dResults={}
with open (pathResults, encoding="utf8") as results:
    for line in results:
        data = line.split(";")
        tweetID=data[0]
        prediction=str(data[1])
        dResults[tweetID]=prediction 
        
distances=[]

for tweetID,predict in dResults.items():
    if tweetID in labels.keys():
        groundTruth=labels[tweetID]
        distances.append(int(round(distance(groundTruth,predict).km)))



lDistances = sorted(distances)
length=len(distances)
index=int(length/2)


print("DeepGeo results:")
avg  = int ( sum(distances)/len(distances))
print("Mean is "+str(avg)+"km")
median = lDistances[index]
print("Median is "+str(median)+"km")
acc_at_161 = 100 * len([d for d in lDistances if d < 161]) / float(len(lDistances))
print("acc@161 is "+str(acc_at_161))